# Example: Graph generation from a gene

This notebook will show you how to use the tool to generate a KG on the underlying data.

In [1]:
import os

current_dir = os.getcwd()

os.chdir(f"{current_dir}/..")

# Set the current working directory
current_dir = os.getcwd()
print("Current directory:", current_dir)

Current directory: /Users/yojana/Documents/GitHub/pyBiodatafuse


In [2]:
# Import modules
import pickle

import pandas as pd

from pyBiodatafuse import id_mapper
from pyBiodatafuse.annotators import (
    bgee,
    disgenet,
    minerva,
    molmedb,
    opentargets,
    pubchem,
    stringdb,
    wikipathways,
)
from pyBiodatafuse.constants import BGEE_GENE_EXPRESSION_LEVELS_COL, DISGENET_DISEASE_COL
from pyBiodatafuse.graph import generator
from pyBiodatafuse.utils import (
    combine_sources,
    create_harmonized_input_file,
    create_or_append_to_metadata,
)

### Load the input list and convert it to a dataframe

In [3]:
genes_of_interest = """AAGRN
ALG14
ALG2
CHAT
CHD8
CHRNA1
CHRNB1
CHRND
CHRNE
CHRNG
COL13A1
COLQ
DOK7
DPAGT1
GFPT1
GMPPB
LAMA5
LAMB2
LRP4
MUSK
MYO9A
PLEC
PREPL
PURA
RAPSN
RPH3A
SCN4A
SLC18A3
SLC25A1
SLC5A7
SNAP25
SYT2
TOR1AIP1
UNC13A
VAMP1
DMD"""

gene_list = genes_of_interest.split("\n")
len(gene_list)

36

In [4]:
data_input = pd.DataFrame(gene_list, columns=["identifier"])
data_input.head()

,identifier
0,AAGRN
1,ALG14
2,ALG2
3,CHAT
4,CHD8


### Entity resolution using BridgeDB

In [5]:
os.makedirs("examples/data", exist_ok=True)

In [6]:
pickle_path = os.path.join(os.getcwd(), "examples/data/example_gene_list.pkl")
metadata_path = os.path.join(os.getcwd(), "examples/data/example_gene_list_metadata.pkl")

if not os.path.exists(pickle_path):
    bridgedb_df, bridgedb_metadata = id_mapper.bridgedb_xref(
        identifiers=data_input,
        input_species="Human",
        input_datasource="HGNC",
        output_datasource="All",
    )
    bridgedb_df.to_pickle(pickle_path)
    with open(metadata_path, "wb") as file:
        pickle.dump(bridgedb_metadata, file)
else:
    bridgedb_df = pd.read_pickle(pickle_path)
    with open(metadata_path, "rb") as file:
        bridgedb_metadata = pickle.load(file)

In [7]:
print("Number of genes with mapping in BridgeDb:", len(bridgedb_df["identifier"].unique()))
bridgedb_df.head()

Number of genes with mapping in BridgeDb: 35


,identifier,identifier.source,target,target.source
0,ALG14,HGNC,Q96F25,Uniprot-TrEMBL
1,ALG14,HGNC,ALG14,HGNC
2,ALG14,HGNC,ENSG00000172339,Ensembl
3,ALG14,HGNC,199857,NCBI Gene
4,ALG14,HGNC,HGNC:28287,HGNC Accession Number


### Gene expression from Bgee

In [10]:
bgee_path = os.path.join(os.getcwd(), "examples/data/example_bgee.pkl")
bgee_metadata_path = os.path.join(os.getcwd(), "examples/data/example_bgee_metadata.pkl")

if not os.path.exists(bgee_path):
    bgee_df, bgee_metadata = bgee.get_gene_expression(bridgedb_df=bridgedb_df)
    bgee_df.to_pickle(bgee_path)
    with open(bgee_metadata_path, "wb") as file:
        pickle.dump(bgee_metadata, file)
else:
    bgee_df = pd.read_pickle(bgee_path)
    with open(bgee_metadata_path, "rb") as file:
        bgee_metadata = pickle.load(file)

In [11]:
bgee_df[BGEE_GENE_EXPRESSION_LEVELS_COL][0]

[{'anatomical_entity_id': 'UBERON_0000178',
  'anatomical_entity_name': 'blood',
  'expression_level': 60.16422,
  'confidence_level_id': 'CIO_0000029',
  'confidence_level_name': 'high confidence level',
  'developmental_stage_id': 'UBERON_0000104',
  'developmental_stage_name': 'life cycle'},
 {'anatomical_entity_id': 'UBERON_0002371',
  'anatomical_entity_name': 'bone marrow',
  'expression_level': 68.7005,
  'confidence_level_id': 'CIO_0000029',
  'confidence_level_name': 'high confidence level',
  'developmental_stage_id': 'UBERON_0000104',
  'developmental_stage_name': 'life cycle'},
 {'anatomical_entity_id': 'UBERON_0000955',
  'anatomical_entity_name': 'brain',
  'expression_level': 73.14512,
  'confidence_level_id': 'CIO_0000029',
  'confidence_level_name': 'high confidence level',
  'developmental_stage_id': 'UBERON_0000104',
  'developmental_stage_name': 'life cycle'},
 {'anatomical_entity_id': 'UBERON_0000310',
  'anatomical_entity_name': 'breast',
  'expression_level': 73.

### Disease annotatation from DisGeNet


In [12]:
disgenet_api_key = "89ba9e26-dc4d-45de-a92d-79fe45d9ae1c"

In [13]:
disgenet_path = os.path.join(os.getcwd(), "examples/data/example_disgenet.pkl")
disgenet_metadata_path = os.path.join(os.getcwd(), "examples/data/example_disgenet_metadata.pkl")

if not os.path.exists(disgenet_path):
    disgenet_df, disgenet_metadata = disgenet.get_gene_disease(
        api_key=disgenet_api_key, bridgedb_df=bridgedb_df
    )

    disgenet_df.to_pickle(disgenet_path)
    with open(disgenet_metadata_path, "wb") as file:
        pickle.dump(disgenet_metadata, file)
else:
    disgenet_df = pd.read_pickle(disgenet_path)
    with open(disgenet_metadata_path, "rb") as file:
        disgenet_metadata = pickle.load(file)

Querying DisGeNET: 100%|██████████| 35/35 [00:16<00:00,  2.15it/s]
/Users/yojana/Documents/GitHub/pyBiodatafuse/src/pyBiodatafuse/annotators/disgenet.py:293: UserWarning: Not all values in column 'el' have the correct type '<class 'float'>'.
  check_columns_against_constants(


### Disease to compound annotation from OpenTargets

In [14]:
# Prepare the input to use DISGENET output as seed for OpenTargets
disease_mapping_df = create_harmonized_input_file(disgenet_df, DISGENET_DISEASE_COL, "EFO", "UMLS")
disease_mapping_df.head()

,identifier,identifier.source,target,target.source
0,UMLS_C5680989,UMLS,EFO_0700079,EFO
2,UMLS_C0017605,UMLS,EFO_1001506,EFO
3,UMLS_C0751884,UMLS,EFO_0700127,EFO
4,UMLS_C0011581,UMLS,EFO_0004257,EFO
5,UMLS_C0011581,UMLS,EFO_0008623,EFO


In [16]:
opentarget_path = os.path.join(os.getcwd(), "examples/data/example_opentarget_cmpd.pkl")
opentarget_metadata_path = os.path.join(
    os.getcwd(), "examples/data/example_opentarget_cmpd_metadata.pkl"
)

if not os.path.exists(opentarget_path):
    opentargets_df, opentargets_metadata = opentargets.get_disease_compound_interactions(
        disease_mapping_df
    )
    opentargets_df.to_pickle(opentarget_path)
    with open(opentarget_metadata_path, "wb") as file:
        pickle.dump(opentargets_metadata, file)
else:
    opentargets_df = pd.read_pickle(opentarget_path)
    with open(opentarget_metadata_path, "rb") as file:
        opentargets_metadata = pickle.load(file)

Mapping PubChem: 100%|██████████| 752/752 [03:28<00:00,  3.60it/s]
/Users/yojana/Documents/GitHub/pyBiodatafuse/src/pyBiodatafuse/annotators/opentargets.py:1119: UserWarning: Not all values in column 'adverse_effect' have the correct type '<class 'dict'>'.
  check_columns_against_constants(


### Pathways from MINERVA

In [17]:
minerva_path = os.path.join(os.getcwd(), "examples/data/example_minerva.pkl")
minerva_metadata_path = os.path.join(os.getcwd(), "examples/data/example_minerva_metadata.pkl")

if not os.path.exists(minerva_path):
    minerva_df, minerva_metadata = minerva.get_gene_minerva_pathways(
        bridgedb_df, map_name="COVID19 Disease Map"
    )
    minerva_df.to_pickle(minerva_path)
    with open(minerva_metadata_path, "wb") as file:
        pickle.dump(minerva_metadata, file)

else:
    minerva_df = pd.read_pickle(minerva_path)
    with open(minerva_metadata_path, "rb") as file:
        minerva_metadata = pickle.load(file)

/var/folders/x2/1tdvwk7x2_n98kzwy7rnt3t00000gn/T/ipykernel_51688/2116567606.py:5: UserWarning: There is no annotation for your input list in MINERVA, project COVID19 Disease Map.
  minerva_df, minerva_metadata = minerva.get_gene_minerva_pathways(


In [18]:
minerva_metadata

{'datasource': 'MINERVA',
 'metadata': {'source_version': '18.0.1'},
 'query': {'size': 35,
  'input_type': 'Ensembl',
  'MINERVA project': 'COVID19 Disease Map',
  'time': '0:01:03.577905',
  'date': '2024-11-06 18:03:00',
  'url': 'https://covid19map.elixir-luxembourg.org/minerva/'}}

### Pathways from WikiPathways

In [19]:
wikipathways_path = os.path.join(os.getcwd(), "examples/data/example_wikipathway.pkl")
wikipathways_metadata_path = os.path.join(
    os.getcwd(), "examples/data/example_wikipathway_metadata.pkl"
)

if not os.path.exists(wikipathways_path):
    wikipathways_df, wikipathways_metadata = wikipathways.get_gene_wikipathways(
        bridgedb_df=bridgedb_df
    )
    wikipathways_df.to_pickle(wikipathways_path)
    with open(wikipathways_metadata_path, "wb") as file:
        pickle.dump(wikipathways_metadata, file)

else:
    wikipathways_df = pd.read_pickle(wikipathways_path)
    with open(wikipathways_metadata_path, "rb") as file:
        wikipathways_metadata = pickle.load(file)

Querying WikiPathways: 100%|██████████| 2/2 [00:01<00:00,  1.30it/s]


### Reactome pathways from OpenTargets

In [20]:
opentargets_reactome_path = os.path.join(os.getcwd(), "examples/data/example_ot_reactome.pkl")
opentargets_reactome_metadata_path = os.path.join(
    os.getcwd(), "examples/data/example_ot_reactome_metadata.pkl"
)

if not os.path.exists(opentargets_reactome_path):
    opentargets_reactome_df, opentargets_reactome_metadata = opentargets.get_gene_reactome_pathways(
        bridgedb_df=bridgedb_df
    )
    opentargets_reactome_df.to_pickle(opentargets_reactome_path)
    with open(opentargets_reactome_metadata_path, "wb") as file:
        pickle.dump(opentargets_reactome_metadata, file)
else:
    opentargets_reactome_df = pd.read_pickle(opentargets_reactome_path)
    with open(opentargets_reactome_metadata_path, "rb") as file:
        opentargets_reactome_metadata = pickle.load(file)

Processing gene-pathway interactions: 100%|██████████| 35/35 [00:00<00:00, 1061.77it/s]


### Gene Ontology from OpenTargets

In [22]:
opentargets_go_path = os.path.join(os.getcwd(), "examples/data/example_ot_go.pkl")
opentargets_go_metadata_path = os.path.join(os.getcwd(), "examples/data/example_ot_go_metadata.pkl")

if not os.path.exists(opentargets_go_path):
    opentargets_go_df, opentargets_go_metadata = opentargets.get_gene_go_process(
        bridgedb_df=bridgedb_df
    )
    opentargets_go_df.to_pickle(opentargets_go_path)
    with open(opentargets_go_metadata_path, "wb") as file:
        pickle.dump(opentargets_go_metadata, file)
else:
    opentargets_go_df = pd.read_pickle(opentargets_go_path)
    with open(opentargets_go_metadata_path, "rb") as file:
        opentargets_go_metadata = pickle.load(file)

Processing gene annotation: 100%|██████████| 35/35 [00:00<00:00, 409.52it/s]


### Compounds from OpenTarget

In [23]:
opentargets_gene_path = os.path.join(os.getcwd(), "examples/data/example_ot_gene_cmpd.pkl")
opentargets_gene_metadata_path = os.path.join(
    os.getcwd(), "examples/data/example_ot_gene_cmpd_metadata.pkl"
)

if not os.path.exists(opentargets_gene_path):
    opentargets_compound_df, opentargets_compound_metadata = (
        opentargets.get_gene_compound_interactions(bridgedb_df=bridgedb_df)
    )
    opentargets_compound_df.to_pickle(opentargets_gene_path)
    with open(opentargets_gene_metadata_path, "wb") as file:
        pickle.dump(opentargets_compound_metadata, file)

else:
    opentargets_compound_df = pd.read_pickle(opentargets_gene_path)
    with open(opentargets_gene_metadata_path, "rb") as file:
        opentargets_compound_metadata = pickle.load(file)

Mapping PubChem: 100%|██████████| 56/56 [00:01<00:00, 44.55it/s]
/Users/yojana/Documents/GitHub/pyBiodatafuse/src/pyBiodatafuse/annotators/opentargets.py:599: UserWarning: Not all values in column 'adverse_effect_count' have the correct type '<class 'int'>'.
  check_columns_against_constants(
/Users/yojana/Documents/GitHub/pyBiodatafuse/src/pyBiodatafuse/annotators/opentargets.py:599: UserWarning: Not all values in column 'adverse_effect' have the correct type '<class 'dict'>'.
  check_columns_against_constants(


### Transporter inhibitors from MolMeDB

In [24]:
molmedb_path = os.path.join(os.getcwd(), "examples/data/example_ot_gene_cmpd.pkl")
molmedb_metadata_path = os.path.join(os.getcwd(), "examples/data/example_ot_gene_cmpd_metadata.pkl")

if not os.path.exists(molmedb_path):
    inhibitor_df, inhibitor_metadata = molmedb.get_gene_compound_inhibitor(bridgedb_df=bridgedb_df)
    inhibitor_df.to_pickle(molmedb_path)
    with open(molmedb_metadata_path, "wb") as file:
        pickle.dump(inhibitor_metadata, file)
else:
    inhibitor_df = pd.read_pickle(molmedb_path)
    with open(molmedb_metadata_path, "rb") as file:
        inhibitor_metadata = pickle.load(file)

### Screening results of compounds on proteins encoded by genes annotation by PubChem

In [25]:
pubchem_path = os.path.join(os.getcwd(), "examples/data/example_pubchem.pkl")
pubchem_metadata_path = os.path.join(os.getcwd(), "examples/data/example_pubchem_metadata.pkl")

if not os.path.exists(pubchem_path):
    pubchem_assay_df, pubchem_assay_metadata = pubchem.get_protein_compound_screened(
        bridgedb_df=bridgedb_df
    )
    pubchem_assay_df.to_pickle(pubchem_path)
    with open(pubchem_metadata_path, "wb") as file:
        pickle.dump(pubchem_assay_metadata, file)

else:
    pubchem_assay_df = pd.read_pickle(pubchem_path)
    with open(pubchem_metadata_path, "rb") as file:
        pubchem_assay_metadata = pickle.load(file)

/var/folders/x2/1tdvwk7x2_n98kzwy7rnt3t00000gn/T/ipykernel_51688/1168625326.py:5: UserWarning: There is no annotation for your input list in PubChem.
  pubchem_assay_df, pubchem_assay_metadata = pubchem.get_protein_compound_screened(


### Protein-Protein interactions from STRING

In [26]:
string_path = os.path.join(os.getcwd(), "examples/data/example_string.pkl")
string_metadata_path = os.path.join(os.getcwd(), "examples/data/example_string_metadata.pkl")

if not os.path.exists(string_path):
    ppi_df, ppi_metadata = stringdb.get_ppi(bridgedb_df=bridgedb_df)
    ppi_df.to_pickle(string_path)
    with open(string_metadata_path, "wb") as file:
        pickle.dump(ppi_metadata, file)
else:
    ppi_df = pd.read_pickle(string_path)
    with open(string_metadata_path, "rb") as file:
        ppi_metadata = pickle.load(file)

/Users/yojana/Documents/GitHub/pyBiodatafuse/src/pyBiodatafuse/annotators/stringdb.py:203: UserWarning: Not all values in column 'score' have the correct type '<class 'int'>'.
  check_columns_against_constants(


### Combing all the results into single dataframe

In [27]:
combined_df = combine_sources(
    bridgedb_df,
    [
        bgee_df,
        disgenet_df,
        minerva_df,
        wikipathways_df,
        opentargets_reactome_df,
        opentargets_go_df,
        opentargets_compound_df,
        inhibitor_df,
        pubchem_assay_df,
        ppi_df,
    ],
)
combined_df.head()

,identifier,identifier.source,target,target.source,Bgee_gene_expression_levels,DISGENET_diseases,WikiPathways,OpenTargets_reactome,OpenTargets_go,OpenTargets_gene_compounds_x,OpenTargets_gene_compounds_y,StringDB_ppi
0,ALG14,HGNC,ENSG00000172339,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...",[{'disease_name': 'Carbohydrate Deficient Glyc...,"[{'pathway_id': 'WP:WP5153', 'pathway_label': ...",[{'pathway_label': 'Biosynthesis of the N-glyc...,"[{'go_id': 'GO:0043541', 'go_name': 'UDP-N-ace...","[{'chembl_id': nan, 'drugbank_id': nan, 'compo...","[{'chembl_id': nan, 'drugbank_id': nan, 'compo...","[{'stringdb_link_to': 'CHRND', 'Ensembl': 'Ens..."
1,ALG2,HGNC,ENSG00000119523,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...",[{'disease_name': 'CONGENITAL DISORDER OF GLYC...,"[{'pathway_id': 'WP:WP5153', 'pathway_label': ...",[{'pathway_label': 'Biosynthesis of the N-glyc...,"[{'go_id': 'GO:0000033', 'go_name': 'alpha-1,3...","[{'chembl_id': nan, 'drugbank_id': nan, 'compo...","[{'chembl_id': nan, 'drugbank_id': nan, 'compo...","[{'stringdb_link_to': 'GMPPB', 'Ensembl': 'Ens..."
2,CHAT,HGNC,ENSG00000070748,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000955', 'a...","[{'disease_name': 'FIMG2, FORMERLY', 'HPO': No...","[{'pathway_id': 'WP:WP5352', 'pathway_label': ...","[{'pathway_label': 'Synthesis of PC', 'pathway...","[{'go_id': 'GO:0004102', 'go_name': 'choline O...","[{'chembl_id': nan, 'drugbank_id': nan, 'compo...","[{'chembl_id': nan, 'drugbank_id': nan, 'compo...","[{'stringdb_link_to': 'SLC5A7', 'Ensembl': 'En..."
3,CHD8,HGNC,ENSG00000100888,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'disease_name': 'AUTISM SPECTRUM DISORDER', ...","[{'pathway_id': 'WP:WP428', 'pathway_label': '...",[{'pathway_label': 'Deactivation of the beta-c...,"[{'go_id': 'GO:0005654', 'go_name': 'nucleopla...","[{'chembl_id': nan, 'drugbank_id': nan, 'compo...","[{'chembl_id': nan, 'drugbank_id': nan, 'compo...","[{'stringdb_link_to': nan, 'Ensembl': nan, 'sc..."
4,CHRNA1,HGNC,ENSG00000138435,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...",[{'disease_name': 'MULTIPLE PTERYGIUM SYNDROME...,"[{'pathway_id': nan, 'pathway_label': nan, 'pa...",[{'pathway_label': 'Highly calcium permeable n...,"[{'go_id': 'GO:0050905', 'go_name': 'neuromusc...","[{'chembl_id': 'CHEMBL:CHEMBL1201248', 'drugba...","[{'chembl_id': 'CHEMBL:CHEMBL1201248', 'drugba...","[{'stringdb_link_to': 'CHRND', 'Ensembl': 'Ens..."


In [28]:
combined_df.shape

(35, 12)

In [29]:
combined_df.to_pickle("examples/data/combined_df.pkl")

In [ ]:
# Shuffling the rows
import numpy as np

combined_df["DISGENET_diseases"] = combined_df["DISGENET_diseases"].apply(np.random.permutation)

In [30]:
combined_metadata = create_or_append_to_metadata(
    bridgedb_metadata,
    [
        bgee_metadata,
        disgenet_metadata,
        opentargets_metadata,
        opentargets_compound_metadata,
        inhibitor_metadata,
        pubchem_assay_metadata,
        ppi_metadata,
        wikipathways_metadata,
        minerva_metadata,
        opentargets_reactome_metadata,
        opentargets_go_metadata,
    ],
)

### Exporting the database in pickle format

In [31]:
combined_df.to_pickle("examples/data/example_df_shuffled.pkl")

with open("example_metadata.pkl", "wb") as out:
    pickle.dump(combined_metadata, out)

with open("opentargets_disease_compound_df.pkl", "wb") as out:
    pickle.dump(opentargets_metadata, out)

## Creating a graph from the annotated dataframe

In [32]:
pygraph = generator.save_graph(
    combined_df=combined_df,
    combined_metadata=combined_metadata,
    disease_compound=opentargets_df,
    graph_name="examples",
    graph_dir="./data",
)

Combined DataFrame saved in ./data/examples/examples_df.pkl
Metadata saved in ./data/examples/examples_metadata.pkl
Building graph: 100%|██████████| 35/35 [00:00<00:00, 1523.46it/s]
Graph is built successfully
Graph saved in ./data/examples/examples_graph.pkl and ./data/examples/examples_graph.gml


#### Cytosacpe
Make sure that the Cytoscape is open

In [ ]:
from pyBiodatafuse.graph import cytoscape

cytoscape.load_graph(pygraph, network_name="Test network")

#### Neo4j

In [37]:
from pyBiodatafuse.graph import neo4j

neo4j.save_graph_to_graphml(pygraph, "networkx_graph_test.graphml")

##### Steps to load the graph in Neo4j

- Add `.graphml` file in **import** subfolder of the DBMS folder
- Install apoc plugin
- Create `apoc.conf` file:
    ```
    apoc.trigger.enabled=true
    apoc.import.file.enabled=true
    apoc.export.file.enabled=true
    apoc.import.file.use_neo4j_config=true
    ```
- Add `apoc.conf` file to **conf** subfolder of the DBMS folder
- Open Neo4j Browser
- (Optionl, only run if you have imported a graph  before) Remove all the nodes before importing `.graphml` file

    ```MATCH (n) DETACH DELETE n```

- Import `.graphml` file

    ```call apoc.import.graphml('file:///networkx_graph_test.graphml',{readLabels:TRUE})```

- Add indexes after importing the graph for improving the performance of queries

    ```
    create index Gene for (n:Gene) on (n.node_type)
    create index Pathway for (n:Pathway) on (n.node_type)
    create index `Biological Process` for (n:`Biological Process`) on (n.node_type)
    create index `Molecular Function` for (n:`Molecular Function`) on (n.node_type)
    create index `Cellular Component` for (n:`Cellular Component`) on (n.node_type)
    create index Disease for (n:Disease) on (n.node_type)
    create index Compound for (n:Compound) on (n.node_type)
    create index `Side Effect` for (n:`Side Effect`) on (n.node_type)
    ```
    

- Count the number of each node type
    - total (```MATCH (n) RETURN count(n)```) 
        - Gene (```MATCH (n:Gene) RETURN count(n)```)
        - Pathway (```MATCH (n:Pathway) RETURN count(n)```)
            - WikiPathways (```MATCH (n:Pathway {source: "WikiPathways"}) RETURN count(n)```) 
            - OpenTargets, Reactome (```MATCH (n:Pathway {source: "OpenTargets"}) RETURN count(n)```) 
            - MINERVA (```MATCH (n:Pathway {source: "MINERVA"}) RETURN count(n)```) 
        - Biological Process (```MATCH (n:`Biological Process`) RETURN count(n)```) 
        - Molecular Function (```MATCH (n:`Molecular Function`) RETURN count(n)```) 
        - Cellular Component (```MATCH (n:`Cellular Component`) RETURN count(n)```) 
        - Disease (```MATCH (n:Disease) RETURN count(n)```) 
        - Compound (```MATCH (n:Compound) RETURN count(n)```)
        - Side Effect (```MATCH (n:`Side Effect`) RETURN count(n)```) 
- Count the number of each edge type
    - total (```MATCH ()-[r]->() RETURN count(r)```) 
        - interacts_with (```MATCH ()-[r:interacts_with]->() RETURN count(r)```) 
        - part_of (```MATCH ()-[r:part_of]->() RETURN count(r)```) 
            - WikiPathways (```MATCH ()-[r:part_of {source: "WikiPathways"}]->() RETURN count(r)```) 
            - OpenTargets, Reactome (```MATCH ()-[r:part_of {source: "OpenTargets"}]->() RETURN count(r)```) 
            - MINERVA (```MATCH ()-[r:part_of {source: "MINERVA"}]->() RETURN count(r)```) 
        - activates (```MATCH ()-[r:activates]->() RETURN count(r)```) 
        - treats (```MATCH ()-[r:treats]->() RETURN count(r)```) 
        - has_side_effect (```MATCH ()-[r:has_side_effect]->() RETURN count(r)```) 
        - inhibits (```MATCH ()-[r:inhibits]->() RETURN count(r)```) = 71
        - associated_with (```MATCH ()-[r:associated_with]->() RETURN count(r)```) 

- Export the graph as a `.csv` file

    ```call apoc.export.csv.all("networkx_graph_test.csv",{})```